## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-20-22-49-29 +0000,nypost,Deadly Legionnaires’ disease outbreak was ‘com...,https://nypost.com/2025/08/20/us-news/deadly-l...
1,2025-08-20-22-40-45 +0000,nyt,"With Moves on West Bank and Gaza City, Israel ...",https://www.nytimes.com/2025/08/20/world/middl...
2,2025-08-20-22-40-00 +0000,wsj,Fed minutes from the July meeting reveal broad...,https://www.wsj.com/economy/central-banking/fe...
3,2025-08-20-22-36-51 +0000,nyt,How Trump Is Forcing Foes to Pay Up,https://www.nytimes.com/2025/08/20/us/politics...
4,2025-08-20-22-34-59 +0000,cbc,U.S. orange juice shipments plummet as Canadia...,https://www.cbc.ca/news/canada/orange-juice-ta...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2352/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
40,trump,41
122,new,20
44,up,12
67,he,12
104,california,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
64,2025-08-20-20-36-51 +0000,nypost,Kremlin’s new demands threaten to derail peace...,https://nypost.com/2025/08/20/us-news/kremlins...,109
27,2025-08-20-21-45-00 +0000,wsj,President Trump has called on a Biden-appointe...,https://www.wsj.com/economy/central-banking/fe...,87
334,2025-08-19-22-54-43 +0000,nypost,Russia launches largest strike on Ukraine in w...,https://nypost.com/2025/08/19/world-news/russi...,86
11,2025-08-20-22-22-41 +0000,nyt,Trump Calls for Fed Governor Lisa Cook to Resi...,https://www.nytimes.com/2025/08/20/us/politics...,86
169,2025-08-20-15-35-01 +0000,nypost,White House trolls Mark Hamill over claim he n...,https://nypost.com/2025/08/20/us-news/white-ho...,82


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
64,109,2025-08-20-20-36-51 +0000,nypost,Kremlin’s new demands threaten to derail peace...,https://nypost.com/2025/08/20/us-news/kremlins...
80,49,2025-08-20-20-14-32 +0000,nypost,"140-pound, 12-foot python captured near South ...",https://nypost.com/2025/08/20/us-news/12-foot-...
27,46,2025-08-20-21-45-00 +0000,wsj,President Trump has called on a Biden-appointe...,https://www.wsj.com/economy/central-banking/fe...
159,46,2025-08-20-16-20-48 +0000,nypost,Mark Zuckerberg’s pricey new AI hires — includ...,https://nypost.com/2025/08/20/business/mark-zu...
119,45,2025-08-20-18-09-26 +0000,nypost,"Israel calls up 60,000 reservists for looming ...",https://nypost.com/2025/08/20/world-news/israe...
2,43,2025-08-20-22-40-00 +0000,wsj,Fed minutes from the July meeting reveal broad...,https://www.wsj.com/economy/central-banking/fe...
28,42,2025-08-20-21-44-31 +0000,nypost,$7.7B Second Avenue subway expansion will put ...,https://nypost.com/2025/08/20/us-news/7-7b-sec...
225,37,2025-08-20-10-00-00 +0000,latimes,Barabak: Power grab may energize Newsom and De...,https://www.latimes.com/politics/story/2025-08...
52,37,2025-08-20-20-57-46 +0000,nypost,"JD Vance, Stephen Miller mock ‘old, white’ pro...",https://nypost.com/2025/08/20/us-news/jd-vance...
239,34,2025-08-20-10-00-00 +0000,latimes,Southern California facing a triple threat: Ex...,https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
